## 한꺼번에 다운받는 데 필요한 처리 내용

지금까지 BeautifulSoup와 CSS 선택자의 사용법을 살펴봤음. 하지만 이것만으로는 링크에 있는 것을 한꺼번에 다운받을 수 없다.

일단 a 태그의 링크 대상이 상대 경로일 수 있음. 그래서 링크 대상이 HTML일 경우, 해당 HTML의 내영에 추가적인 처리를 해야함. 그리고 링크를 재귀적으로 다운받아야함.

이번 절에서는 링크에 있는 것을 한꺼번에 다운받는 기법을 소개하겠음.

## 상대 경로를 전개하는 방법

그림 일단 첫 번째 문제부터 살펴보자.

일단 a 태그의 href 속성에 링크대상이 ".../img/hoge.png"처럼 상대 경로로 적혀 있다고 하자.
a 태그가 상대 경로로 주어졌을 때 대상에 있는 것을 다운받으려면 상대 경로를 절대 경로로 변환해야함.

상대 경로를 전개할 때는 urllib.parse.urljoin()을 사용해야함. 

In [1]:
from urllib.parse import urljoin

base = "http://example.com/html/a.html"

print( urljoin(base, "b.html"))
print( urljoin(base, "sub/c.html"))
print( urljoin(base, "../index.html"))
print( urljoin(base, "../img/hogo.png"))
print( urljoin(base, "../css/hogo.css"))

http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/img/hogo.png
http://example.com/css/hogo.css


결과를 보면 기본 URL 기반으로 상대 경로를 절대 경로로 변환한다는 것을 알 수 있음. 
이처럼 상대 경로를 절대 경로로 변환하는 urljoin() 함수의 사용법을 살펴보자

urllib.parse.urljoin() 사용법
urljoin(bast, path)

이 함수는 첫 번째 매개변수로 기본 URL, 두 번째 매개변수로 상대 경로를 지정함.

만약 상대 경로(path 매개변수)가 http:// 등으로 시작한다면 기본 URL(bsse 매개변수)을 무시하고 두 번쨰 매개변수에 지정한 URL을 리턴함. 

예제코드를 보자.

In [2]:
print( urljoin(base, "hogo.html"))
print( urljoin(base, "http://otherExample.com/wiki"))
print( urljoin(base, "//anotherExample.org/test"))

http://example.com/html/hogo.html
http://otherExample.com/wiki
http://anotherExample.org/test


이처럼 urljoin() 함수를 사용하면 a 태그의 href 속성에 지정돼 있는 경로를 절대 경로로 쉽게 변환가능.

## 모든 페이지를 한꺼번에 다운받는 프로그램

그럼 이를 구현한 프로그램을 살펴보자. 일단 이번 절에서는 웹에 있는 파이썬 문서 중에서 library 폴더 아래에 있는 모든 것을 다운받아 보겠음.



In [3]:
# 파이썬 매뉴얼을 재귀적으로 다운받는 프로그램
# 모듈 읽어 들이기 - (1)
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re

# 이미 처리한 파일인지 확인하기 위한 변수 - (2)
proc_files = {}

# HTML 내부에 있는 링크를 추출하는 함수 - (3)
def enum_links(html, base) :
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']") # CSS
    links += soup.select("a[href]") # 링크
    result = []
    
    # href 속성을 추출하고, 링크를 절대 경로로 변환 - (4)
    for a in links:
        href = a.ttrs['href']
        url = urljoin(base, href)
        result.append(url)
        return result
    
    # 파일을 다운받고 저장하는 함수 - (5)
    def download_file(url):
        o = urlparse(url)
        savepath = "./" + o.netloc + o.path
        if re.search(r"/$", savepath): # 폴더라면 index.html
            savepath += "index.html"
        savedir = os.path.dirname(savepath)
        
        # 모두 다운됐는지 확인
        if os.path.exists(savepath): return savepath
        
        # 다운받을 폴더 생성
        if not os.path.exists(savedir):
            print("mkdir = ", savedir)
            makedirs(savedir)
            
        # 파일 다운받기 - (6)
        try :
            print("download = ", url)
            urlretrieve(url, savepath)
            time.sleep(1) # 1초 휴식 - (7)
            return savepath
        except:
            print("다운 실패:", url)
            return none
        
        # HTML을 분석하고 다운받는 함수 - (8)
        def analyze_html(url, root_url):
            savepath = download_file(url)
            
            if savepath in None : return
            if savepath is proc_files: return # 이미 처리 했다면 처리하지 않음 - (9)
            proc_files[savepath] = True
            print("analyze_html", url)
            
        # 링크 추출 - (10)
        html = open(savepath, "r", encoding="utf-8").read()
        links = enum_links(html, url)
        
        for link_url in links:
            #링크가 루트 이외의 경로를 나타내면 무시 - (11)
            if link_url.find(root_url) != 0:
                if not re.search(r".css$",link_url): continue
            # HTML 이라면
            if re.search(r".(html|html)$", link_url):
                # 재귀적으로 HTML 파일 분석하기
                analyze_html(link_url, root_url)
                continue
            # 기타파일
            dwonload_file(link_url)
            
            # URL에 있는 모든 것 다운로드 받기
            if __name__ == "__name__":
                url = "https://docs.python.org/3/"
                analyze_html(url, url)